# <center> Контрольна робота  №1 та №2 (ОДЗ)


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.regression module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


У завданні буде використано набір даних про якість білого вина(репозиторій UCI)
archive.ics.uci.edu/ml/machine-learning-databases/wine-quality.
Завантажте дані

In [2]:
!mkdir -p data
!wget -P data -c https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

data = pd.read_csv("data/winequality-white.csv", sep=";")
display(data.sample(10))

--2020-12-21 09:07:21--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264426 (258K) [application/x-httpd-php]
Saving to: ‘data/winequality-white.csv’

winequality-white.c 100%[===================>] 258.23K  1007KB/s    in 0.3s    

2020-12-21 09:07:21 (1007 KB/s) - ‘data/winequality-white.csv’ saved [264426/264426]



,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1269,8.4,0.35,0.56,13.8,0.048,55.0,190.0,0.99930,3.07,0.58,9.400000,6
4353,7.4,0.16,0.30,1.4,0.064,34.0,166.0,0.99136,3.11,0.42,11.433333,6
4508,5.8,0.26,0.30,2.6,0.034,75.0,129.0,0.99020,3.20,0.38,11.500000,4
2810,7.8,0.26,0.31,3.6,0.025,22.0,100.0,0.99066,2.99,0.47,12.100000,7
4834,5.9,0.37,0.30,1.5,0.033,35.0,95.0,0.98986,3.36,0.56,12.000000,7
213,6.7,0.13,0.29,5.3,0.051,31.0,122.0,0.99440,3.44,0.37,9.700000,6
2851,7.3,0.23,0.34,9.3,0.052,19.0,86.0,0.99574,3.04,0.56,10.000000,5
4592,6.7,0.45,0.30,5.3,0.036,27.0,165.0,0.99122,3.12,0.46,12.200000,6
2456,8.1,0.33,0.36,7.4,0.037,36.0,156.0,0.99592,3.19,0.54,10.600000,6
1806,7.6,0.38,0.20,3.4,0.046,9.0,116.0,0.99440,3.15,0.41,9.400000,5


In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Відокремте цільову змінну, розділіть навчальну вибірку у відношенні 7:3 (30% - під задишену вибірку, нехай random_state=17) і нормалізуйте дані за допомогою StandartScaler


In [5]:
y = data["quality"]
data.drop("quality", axis=1, inplace=True)
X_train, X_holdout, y_train, y_holdout = train_test_split(data, y, train_size=0.7, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Лінійна регресія


Навчіть просту лінійну модель регресії

In [6]:
linreg = LinearRegression(n_jobs=-1)
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

> **Питання 1 : Які середньоквадратичні помилки лінійної регресії на навчальній і відкоаденій вибірках ?**

In [7]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" %  mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


Подивіться на коефіцієнти моделі і ранжуйте ознаки за впливом на якість вина (врахуйте, що великі за модулем негативні значення коефіцієнтів теж говорять про сильний вплив). Створіть для цього новий невеликий DataFrame.
> **Питання 2 : Яку ознаку лінійна регресія вважає найбільш впливовою на якість вина?**




In [8]:
linreg_coef = pd.DataFrame(linreg.coef_, index=X_train.columns, columns=["Coefficient"])
linreg_coef.sort_values(by="Coefficient", key=lambda x: abs(x), ascending=False, inplace=True)
linreg_coef

,Coefficient
density,-0.665720
residual sugar,0.538164
volatile acidity,-0.192260
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127


## Lasso-регресія

**Навчіть Lasso-регресію з невеликим коефіцієнтом alpha=0,01 (слабка регуляризація). Нехай знову random_state=17.**


In [9]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=17,
      selection='cyclic', tol=0.0001, warm_start=False)

**Подивіться на коефіцієнти моделі і ранжуйте ознаки за впливом на якість вина. Яка ознака "відпала" першою, тобто найменш важлива для пояснення цільової змінної в моделі Lasso?**

In [13]:
lasso1_coef = pd.DataFrame(lasso1.coef_, index=X_train.columns, columns=["Coefficient"])
lasso1_coef.sort_values(by="Coefficient", key=lambda x: abs(x), ascending=True, inplace=True)
lasso1_coef

,Coefficient
fixed acidity,-0.000000
citric acid,-0.000000
total sulfur dioxide,-0.000000
chlorides,-0.002747
sulphates,0.029722
free sulfur dioxide,0.043088
pH,0.067277
volatile acidity,-0.188479
density,-0.235492
residual sugar,0.256363


**Тепер визначте краще значення alpha в процесі 5-кратної крос-валідації. Використовуйте LassoCV і random_state = 17.**

In [11]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17, n_jobs=-1)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=-1, normalize=False, positive=False,
        precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
        verbose=False)

In [12]:
lasso_cv.alpha_

0.0002833096101839324

Виведіть коефіцієнти "кращого" Lasso в порядку зменшення впливу на якість вина.
> **Питання 3: Яка ознака "занулилася першою" в налаштованій моделі LASSO?**

In [15]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_, index=X_train.columns, columns=["Coefficient"])
lasso_cv_coef.sort_values(by="Coefficient", key=lambda x: abs(x), ascending=False, inplace=True)
lasso_cv_coef

,Coefficient
density,-0.648161
residual sugar,0.526883
volatile acidity,-0.192049
pH,0.146549
alcohol,0.137115
fixed acidity,0.093295
sulphates,0.060939
free sulfur dioxide,0.042698
total sulfur dioxide,0.012969
chlorides,0.006933


**Оцініть середньоквадратичну помилку моделі на навчальній і тестовій вибірках.**

> **Питання 4 : Які середньоквадратичні помилки налаштованої LASSO-регресії на навчальній і відкладеній вибірках?**

In [17]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Випадковий ліс

**Навчіть випадковий ліс з параметрами "з коробки", фіксуючи тільки random_state=17.**

In [18]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

> **Питання 5 : Які середньоквадратичні помилки випадкового лысу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error' і іншими параметрами за замовчуванням) і відкладеній вибірках?**

In [28]:
print("Mean squared error (train): %.3f" %  mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % cross_val_score(estimator=forest,
                                                        X=X_train_scaled,
                                                        y=y_train, scoring="neg_mean_squared_error", n_jobs=-1)[0])
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.053
Mean squared error (cv): -0.456
Mean squared error (test): 0.372


**Налаштуйте параметри min_samples_leaf і max_depth за допомогою GridSearchCV і знову перевірте якість моделі на крос-валідації і на відкладеній вибірках.**

In [30]:
forest_params = {'max_depth': list(range(10, 25)),
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6, 12))}
locally_best_forest = GridSearchCV(RandomForestRegressor(random_state=17), forest_params, n_jobs=-1)
locally_best_forest.fit(X_train_scaled, y_train)


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=17,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_j

In [31]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 0.4939384585901731)

**Нажал результати GridSearchCV в повному не відтворювані (можуть відрізнятися на різних платформах навіть при фіксованому random_state). Тому навчіть ліс з параметрами max_depth=19, max_features=7, i min_samples_leaf=1 (краще в моэму випадку).**
> **Питання 6 : Які середньоквадратичні помилки налаштованого випадкового лісу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error') і на відкладеній вибірках?**


In [32]:
# create model with {max_depth=19, max_features=7, i min_samples_leaf=1} params
best_forest_regressor = RandomForestRegressor(random_state=17, max_depth=19, max_features=7, min_samples_leaf=1)
best_forest_regressor.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=19, max_features=7, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

In [33]:
print("Mean squared error (cv): %.3f" % mean_squared_error(y_train, best_forest_regressor.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, best_forest_regressor.predict(X_holdout_scaled)))

Mean squared error (cv): 0.058
Mean squared error (test): 0.370


**Оцініть важливість ознак за допомогою випадкового лісу.**
>**Питання 7 : Яка ознака виявилася найінформативнішою в налаштованій моделі випадкового лісу?**

In [34]:
rf_importance = pd.DataFrame(best_forest_regressor.feature_importances_, index=X_train.columns, columns=["Importance"])
rf_importance.sort_values(by="Importance", ascending=False, inplace=True)
rf_importance

,Importance
alcohol,0.214825
volatile acidity,0.117627
free sulfur dioxide,0.113509
density,0.086530
pH,0.072646
residual sugar,0.072529
total sulfur dioxide,0.071644
chlorides,0.067819
fixed acidity,0.061873
citric acid,0.061500


**Зробіть висновки про якість моделей і оцінки впливу ознак на якість вина за допомогою цих трьох моделей**

У даній роботі було розглянуто моделі лінійної регресії, Lasso-регресії та випадковий ліс. За метрику якості було взято середньоквадратичну помилку. Перші дві моделі показали вищу помилку на тестових даних, ніж на тренувальних. Але при цьому Модель випадкового лісу показала меншу середньоквадратичну помилку, що свідчить про кращі узагальнюючі властивості моделі.
Також на якість моделей впливає важливість ознак: так, наприклад, ознака 'alcohol', що є найбільш важливою для випадкового лісу, займає невисоке положення у важливості для лінійної регресії та Lasso-регресії. Враховуючи, що випадковий ліс має краще значення метрики, можемо сказати, що його порядок важливості ознак ближчий до істиннного.